In [1]:
import os, pathlib, shutil
from collections import Counter
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Subset
from torchvision.transforms import v2
import torch.nn as nn
import numpy as np
import torch
from torch.profiler import profile, record_function, ProfilerActivity
import train
from torchvision.transforms import InterpolationMode
from torchvision.datasets import ImageFolder

src = ""
if os.name == "nt":
    src = "D:/599DL4VProject/the_wildfire_dataset"
elif os.name == "posix":
    src = "/home/asromelo/Desktop/Projects/599_proj/the_wildfire_dataset/"

wf1TrainPath = pathlib.Path(src + '/train').resolve()
wf1ValidPath = pathlib.Path(src + '/valid').resolve()
wf1TestPath = pathlib.Path(src + '/test').resolve()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cuda = True if torch.cuda.is_available() else False

if device == 'cuda':
    torch.backends.cudnn.benchmark = True
else:
    torch.jit.onednn_fusion_enabled = True

# Currently using Kaggle The Wildfire Dataset and FiSmo dataset
# https://www.kaggle.com/datasets/utkarshsaxenadn/landscape-recognition-image-dataset-12k-images
# https://www.kaggle.com/datasets/uciml/forest-cover-type-dataset

c:\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Python311\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorc

In [2]:
p = 0.2
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# Valid img size is 128, 160, 192, 224
img_size = 128

tfset = v2.Compose([
        v2.ToImageTensor(),
        v2.ConvertImageDtype(torch.uint8),
        v2.RandomCrop(size=512, pad_if_needed=True, padding_mode='edge'),
        v2.RandomChoice(transforms=[
            v2.Grayscale(3),
            v2.ColorJitter(p,p,p,p),
            v2.RandomEqualize(p),
            v2.RandomHorizontalFlip(p)
        ], p=[0.5,0.3,0.5,0.5]), 
        v2.Resize(size=img_size, antialias=False),
        v2.ConvertImageDtype(dtype=torch.float),
])

testset = v2.Compose([
    v2.ToImageTensor(),
    v2.ConvertImageDtype(torch.uint8),
    v2.Resize(size=720, interpolation=InterpolationMode.BILINEAR, antialias=True),
    v2.CenterCrop(size=512),
    v2.Resize(size=img_size, antialias=False),
    v2.ConvertImageDtype(dtype=torch.float)
])

wf1Train = train.ForestFireDataset(root=str(wf1TrainPath), transform=tfset)
wf1Valid = train.ForestFireDataset(root=str(wf1ValidPath), transform=testset)
wf1Test = train.ForestFireDataset(root=str(wf1TestPath), transform=testset)

wf1Tsfire = Subset(wf1Test, [i for i, label in enumerate(wf1Test.targets) if label == 0])
wf1Tsnofire = Subset(wf1Test, [i for i, label in enumerate(wf1Test.targets) if label == 1])
wf1nofire = Subset(wf1Train, [i for i, label in enumerate(wf1Train.targets) if label == 1])

batch_size = 8
wf1TrLoader = DataLoader(dataset=wf1nofire, batch_size=batch_size, shuffle=True, num_workers=2)
wf1VaLoader = DataLoader(dataset=wf1Valid, batch_size=batch_size, num_workers=2)
wf1TsLoader = DataLoader(dataset=wf1Test, batch_size=batch_size, num_workers=2)
wf1TsFireLd = DataLoader(dataset=wf1Tsfire, batch_size=batch_size, num_workers=2)
wf1TsNoFireLd = DataLoader(dataset=wf1Tsnofire, batch_size=batch_size, num_workers=2)

In [3]:
epochs = 5
trainer = train.GANProject(img_size=img_size, debug=False, small=True, activation=nn.LeakyReLU(0.1), alternative="efficientnet_v2_s")

c:\Python311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [4]:
# Next W = (W + 2*pad - Kernel)/stride + 1
w=128 + 32
k=3
p=1
s=1

def downlayer(w,k,p,s):
    a1 = conv(w,k,p,s)
    a2 = conv(a1,k,p,s)
    return conv(a2,2,0,2)

def conv(w,k,p,s):
    return (w + 2*p - k)/s + 1

def unpooled(w,k,p,s):
    h = (w + 2*p - k)/s + 1
    return (h + 2*p - k)/s + 1

def backward(w,k,p,s):
    f = 2*(w - 1) + 2 - 2*p
    g = s*(f - 1) + k - 2*p
    return s*(g - 1) + k - 2*p

def uplayer(w,k,p,s):
    d = 1
    innerkernel = 2
    innerstride = 2
    innerpad = 0
    outpad = 0
    # Formula for Conv = (H_in - 1)*stride - 2*p + dilation*(k-1) + outpad + 1
    u = innerstride*(w - 1) - 2*innerpad + d*(innerkernel-1) + outpad + 1
    a1 = conv(u,k,p,s)
    a2 = conv(a1,k,p,s)
    return a2
    # 148 -> 72 -> 34 -> 30

a = downlayer(w,k,p,s)
b = downlayer(a,k,p,s)
c = downlayer(b,k,p,s)
bottom = unpooled(c,k,p,s)
d = uplayer(bottom,k,p,s)
e = uplayer(d,k,p,s)
f = uplayer(e,k,p,s)
conv(f,1,0,1)

160.0

In [5]:
trainer.train_model(trainLoader=wf1TrLoader, validLoader=wf1VaLoader, epochs=epochs, eps=0.01)

c:\Python311\Lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 0 Time: 212.20s
Train:  {'rec_loss': 0.09379781705630225, 'd_loss': 1.6265230961483663, 'ec_loss': 0, 'feature_loss': 0.771478438449848}
Validation:  {'rec_loss': 0.056281678119109044, 'd_loss': 1.6508001071303637, 'ec_loss': 0.003381366160378527, 'accuracy': 8.0}
Epoch 1 Time: 219.58s
Train:  {'rec_loss': 0.0831479301568585, 'd_loss': 1.6265230961483663, 'ec_loss': 0, 'feature_loss': 1.5029122488839286}
Validation:  {'rec_loss': 0.06330721651143696, 'd_loss': 1.6508001071303637, 'ec_loss': 0.003756063494516249, 'accuracy': 8.0}
Epoch 2 Time: 213.87s
Train:  {'rec_loss': 0.07708621967164941, 'd_loss': 1.6265230961483663, 'ec_loss': 0, 'feature_loss': 2.776107277913659}
Validation:  {'rec_loss': 0.08151637025140411, 'd_loss': 1.6508001071303637, 'ec_loss': 0.002462547068572163, 'accuracy': 8.0}
Epoch 3 Time: 208.25s
Train:  {'rec_loss': 0.07396746116568614, 'd_loss': 1.6265230961483663, 'ec_loss': 0, 'feature_loss': 4.108897107712766}
Validation:  {'rec_loss': 0.05758493812523078,